In [9]:
# import regular expressins packge
# import numbers package
import numpy as np
import re

In [10]:
#read file
def readFile(fileName):
    file = open(fileName,'r',encoding="cp437")
    fileText = ""
    for line in file:
        fileText += line
    return fileText

In [11]:
# preprocess text
def preProcess(text):
    # Remove non-letter chars
    text = re.sub("[^a-zA-Z ]"," ", text)
    # Change characters to lower
    text = text.lower()

    text = " ".join(word for word in text.split() if len(word) > 2)
    return text

In [12]:
def genReferenceWordList(texts,stopWords):
# concatenate the texts
    allText = ""
    for line in texts:
        allText += line
    # Generate a word list
    wordsList =  allText.split()
    # Generate a word set
    wordsSet =  set(wordsList)
    # Remove the stop words from the word list
    stopWordsList = stopWords.split()
    stopWordsSet = set(stopWordsList)
    refWordSet = wordsSet.difference(stopWordsSet)
    return list(refWordSet)

In [13]:
# vector distance
# find the distance between arrays
def vecDist(vec1,vec2):
    vecDiff = vec1-vec2
    # compute the distance ("pitagoras")
    vecSqr =  np.square(vecDiff)
    vecSum =  np.sum(vecSqr)
    return np.sqrt(vecSum)

In [14]:
def vecDist_New(vec1, vec2):
    vecDiff = vec1 - vec2
    # compute the distance (sqrt of sum of absolute differences)
    vecAbs = np.abs(vecDiff)
    vecSum = np.sum(vecAbs)
    return np.sqrt(vecSum)

In [15]:
# word frequency
def getWordFrequency(texts,dictList):
    dictSize = len(dictList)
    nTexts = len(texts)
    wordFreq = np.empty((nTexts,dictSize))
    for i in range(nTexts):
        print("text" + str(i))
        for j in range(dictSize):
            wordFreq[i,j] = texts[i].count(dictList[j])
    return wordFreq

In [16]:
books = []
#read  and preprocess files
books+= [readFile('DB.txt')]
books+= [readFile('Eliot.txt')]
books+= [readFile('HP.txt')]
books+= [readFile('Tolkien.txt')]

In [17]:
window = 150000

In [18]:
# split every book into two parts
bookParts = []
for book in books:
    nParts = len(book)//window
    for i in range(nParts):
        bookParts += [book[i*window:(i+1)*window]]

In [19]:
# preprocess texts in book parts
texts = []
for text in bookParts:
    texts += [preProcess(text)]

In [20]:
# Read stop words file - words that can be removed
stopWords = readFile('stopwords_en.txt')

In [21]:
# generate reference word list
refList = genReferenceWordList(texts,stopWords)

In [22]:
# Find the frequency of the reference words in the files
wordFreq = getWordFrequency(texts,refList)

text0
text1
text2
text3
text4
text5
text6
text7
text8
text9
text10
text11


In [23]:
# find the distance matrix between the text files
# text distances
rows,colomns = wordFreq.shape

dist = np.zeros((rows,rows))
for i in range(rows):
    for j in range(rows):
        # calculate the distance between the frequency vectors
        dist[i,j] = round(vecDist(wordFreq[i],wordFreq[j]))

print("dist matrix = \n",dist)

dist matrix = 
 [[   0.  611.  703. 1098. 1024. 1025. 1343. 1309. 1264. 1130. 1221. 1197.]
 [ 611.    0.  534. 1093. 1000. 1000. 1310. 1289. 1236. 1097. 1180. 1158.]
 [ 703.  534.    0. 1097. 1006. 1014. 1355. 1327. 1272. 1124. 1222. 1204.]
 [1098. 1093. 1097.    0.  520.  519. 1273. 1211. 1175.  949. 1054. 1034.]
 [1024. 1000. 1006.  520.    0.  477. 1210. 1167. 1110.  893. 1023.  995.]
 [1025. 1000. 1014.  519.  477.    0. 1207. 1155. 1103.  889.  997.  960.]
 [1343. 1310. 1355. 1273. 1210. 1207.    0.  592.  646. 1297. 1298. 1260.]
 [1309. 1289. 1327. 1211. 1167. 1155.  592.    0.  652. 1252. 1272. 1246.]
 [1264. 1236. 1272. 1175. 1110. 1103.  646.  652.    0. 1190. 1212. 1176.]
 [1130. 1097. 1124.  949.  893.  889. 1297. 1252. 1190.    0.  677.  667.]
 [1221. 1180. 1222. 1054. 1023.  997. 1298. 1272. 1212.  677.    0.  530.]
 [1197. 1158. 1204. 1034.  995.  960. 1260. 1246. 1176.  667.  530.    0.]]


In [24]:
xxx_raw_text = readFile('xxx.txt')
xxx_processed_text = preProcess(xxx_raw_text)
print("Length of raw text: %d" % len(xxx_raw_text))
print("Length of processed text: %d" % len(xxx_processed_text))

Length of raw text: 260892
Length of processed text: 219150


In [ ]:
xxx_wordFreq = getWordFrequency([xxx_processed_text], refList)
print(f"Shape of xxx_wordFreq: {xxx_wordFreq.shape}")

text0


In [ ]:
distances_to_xxx = []
num_book_parts = wordFreq.shape[0]

for i in range(num_book_parts):
    # vecDist function expects two vectors, xxx_wordFreq is (1, N) so we take its first row
    distance = vecDist(wordFreq[i], xxx_wordFreq[0])
    distances_to_xxx.append(round(distance))

print("Distances from xxx.txt to each book part:", distances_to_xxx)

In [ ]:
min_distance = min(distances_to_xxx)
min_distance_index = distances_to_xxx.index(min_distance)

# Assuming each book was split into `my_nParts` (which is 3 from the context)
book_titles = ['DB.txt', 'Eliot.txt', 'HP.txt', 'Tolkien.txt']
book_part_size = 3 # from my_nParts which was len(book) // my_window (150000)

# Determine which original book the closest part belongs to
closest_book_index = min_distance_index // book_part_size
closest_book_title = book_titles[closest_book_index]

print("The minimum distance to xxx.txt is: %d" % min_distance)
print("This distance corresponds to book part index: %d" % min_distance_index)
print("The original book most similar to xxx.txt is: %s" % closest_book_title)